##### Saving words from newspaper titles in textfiles

In [2]:
import json

# Path to the JSON file
json_file_path = 'cdx_results_2019.json'

# Open and load the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# List to store the URLs
urls = []

# Loop through the data and extract 'url' values
for entry in data:
    if 'url' in entry:  # Check if 'url' key exists
        urls.append(entry['url'])  # Add the URL to the list

# Open a file to write the results
with open("titles.txt", "w") as file:
    # Loop through each URL
    for url in urls:
        # Remove 'https://' and split the URL at slashes
        parts = url.split('/')
        
        # Extract the last part after the slash, which usually contains the title
        title_with_params = parts[-1]
        
        # Remove any parameters (everything after '?')
        title = title_with_params.split('?')[0]
        
        # Replace hyphens with spaces to improve readability
        formatted_title = title.replace('-', ' ')
        
        # Write the formatted title to the file
        file.write(f"{formatted_title}\n")

print("Titles extracted and written to 'titles.txt'.")

# maybe store urls and words wich belong togethers in a dictionary

Titles extracted and written to 'titles.txt'.


In [3]:
import json

# Function to split the URL into words from the title
def split_url_in_words(url):
    # Remove 'https://' and split the URL at slashes
    parts = url.split('/')
    
    # Extract the last part after the slash, which usually contains the title
    title_with_params = parts[-1]
    
    # Remove any parameters (everything after '?')
    title = title_with_params.split('?')[0]
    
    # Replace hyphens with spaces to improve readability
    formatted_title = title.replace('-', ' ')
    
    # Return the split words
    return formatted_title

# Path to the JSON file
json_file_path = 'cdx_results_2019.json'

# Open and load the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Empty dictionary to store the URLs as keys and the split title words as values
urls = {}

# Iterate through the JSON data
for entry in data:
    if 'url' in entry:
        #url = entry['url']  # Get the full URL
        #split_title = split_url_in_words(url)  # Split the URL title into words
        #urls[str(entry['url'])] = split_title  # Store the URL as key and split title as value
        urls[str(entry['url'])] = split_url_in_words(entry['url'])

# Writing dictionary to a textfile
with open('urls_titles.json', 'w') as output_file:
    json.dump(urls, output_file, indent=4)

 Trying to scrape only the < title > elements in the links and look up if Chega is written as: 'Chega' or 'CHEGA':

In [4]:
import json
import requests
from bs4 import BeautifulSoup
import time

# Path to the JSON file
file_name = 'test_for_scraping_cdx_results.json'

# Open and load the JSON file
with open(file_name, 'r') as file:
    data = json.load(file)

titles = {}

# Loop through URLs; search for the title element in HTML-Script
for v in data:
    url = v['url']
    
    try:
        # Requesting the website and setting a timeout of 10 seconds
        response = requests.get(url, timeout=10, stream=True) # stream=True for downloading data in chunks not everything at once 
        
        # Delay between requests to avoid overloading the server
        time.sleep(1)  # 1-second delay

        # Initialize an empty content variable and stream content chunks
        html_content = b""
        
        # Stream the content until we find the closing </title> tag
        for chunk in response.iter_content(chunk_size=512):
            html_content += chunk
            if b"</title>" in html_content:
                break  # Stop streaming once the <title> tag is found

        # Parse only the partial content with BeautifulSoup
        soup = BeautifulSoup(html_content, "html.parser")

        # Extract the <title> element
        title_tag = soup.find("title")

        # If no <title> tag is found, skip this page, makes program faster
        if not title_tag:
            continue

        # Extract the text from the <title> tag
        title_text = title_tag.get_text()

        # Check for "Chega" or "CHEGA" (case-sensitive check)
        if "Chega" in title_text or "CHEGA" in title_text:
            titles[url] = title_text  # Store the URL and title in the dictionary

    except requests.exceptions.Timeout:
        print(f"Timeout occurred for URL: {url}")
        continue  # Skip to the next URL if a timeout occurs

    except requests.exceptions.RequestException as e:
        print(f"Request failed for URL: {url} with error: {e}")
        continue  # Skip to the next URL if another error occurs

# Writing the dictionary to a JSON file
with open('titles.json', 'w', encoding='utf-8') as json_file:
    json.dump(titles, json_file, indent=4, ensure_ascii=False)

# Print the results, only for debugging, to remove later
for url, title in titles.items():
    print(url, title)

Timeout occurred for URL: https://www.jn.pt/ntv/interior/lourenco-ortigao-e-kelly-bailey-dao-tudo-e-mais-alguma-coisa-mas-nao-chega-10616299.html
https://www.cmjornal.pt/politica/amp/andre-ventura-diz-chega-vai-impedir-extrema-direita-em-portugal André Ventura diz que Chega vai impedir extrema-direita em Portugal - Política - Correio da Manhã
https://www.cmjornal.pt/politica/amp/partido-chega-de-andre-ventura-inicia-formalizacao-para-ser-alternativa-a-direita-que-parece-nao-existir Partido 'CHEGA' inicia formalização para ser alternativa "à direita que parece não existir"  - Política - Correio da Manhã


In [5]:
# Debug output:

for url, title in titles.items():
    print(f'{url}: {title}')

https://www.cmjornal.pt/politica/amp/andre-ventura-diz-chega-vai-impedir-extrema-direita-em-portugal: André Ventura diz que Chega vai impedir extrema-direita em Portugal - Política - Correio da Manhã
https://www.cmjornal.pt/politica/amp/partido-chega-de-andre-ventura-inicia-formalizacao-para-ser-alternativa-a-direita-que-parece-nao-existir: Partido 'CHEGA' inicia formalização para ser alternativa "à direita que parece não existir"  - Política - Correio da Manhã


In [6]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Combine all titles into one string
all_titles = ' '.join(titles.values())

# Generate a word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_titles)

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  # No axes for a clean look
plt.show()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/hannes_seidl/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/hannes_seidl/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/hannes_seidl/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/hannes_seidl/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in la

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [1]:
import json
import requests
from bs4 import BeautifulSoup
import time

# Path to the JSON file
file_name = 'cdx_results_2019.json'

# Open and load the JSON file
with open(file_name, 'r') as file:
    data = json.load(file)

titles = {}

# Loop through URLs; search for the title element in HTML-Script
counter = 0

for v in data:
    url = v['url']
    print(f'iteration {counter}')
    print(f'link: {url}')
    counter += 1
    
    try:
        # Requesting the website and setting a timeout of 10 seconds
        response = requests.get(url, timeout=10, stream=True) # stream=True for downloading data in chunks not everything at once 
        
        # Delay between requests to avoid overloading the server
        #time.sleep(1)  # 1-second delay

        # Initialize an empty content variable and stream content chunks
        html_content = b""
        
        # Stream the content until we find the closing </title> tag
        for chunk in response.iter_content(chunk_size=512):
            html_content += chunk
            if b"</title>" in html_content:
                break  # Stop streaming once the <title> tag is found

        # Parse only the partial content with BeautifulSoup
        soup = BeautifulSoup(html_content, "html.parser")

        # Extract the <title> element
        title_tag = soup.find("title")

        # If no <title> tag is found, skip this page, makes program faster
        if not title_tag:
            continue

        # Extract the text from the <title> tag
        title_text = title_tag.get_text()

        # Check for "Chega" or "CHEGA" (case-sensitive check)
        if "Chega" in title_text or "CHEGA" in title_text:
            print(f'title text "{title_text} is valid"')
            titles[url] = title_text  # Store the URL and title in the dictionary

    except requests.exceptions.Timeout:
        print(f"Timeout occurred for URL: {url}")
        continue  # Skip to the next URL if a timeout occurs


    except requests.exceptions.RequestException as e:
        print(f"Request failed for URL: {url} with error: {e}")
        continue  # Skip to the next URL if another error occurs

# Writing the dictionary to a JSON file
with open('titles.json', 'w', encoding='utf-8') as json_file:
    json.dump(titles, json_file, indent=4, ensure_ascii=False)

iteration 0
link: https://www.cmjornal.pt/boa-vida/amp/capital-do-natal-chega-a-oeiras
iteration 1
link: https://www.cmjornal.pt/boa-vida/amp/capital-do-natal-chega-a-oeiras
iteration 2
link: https://www.cmjornal.pt/boa-vida/detalhe/capital-do-natal-chega-a-oeiras
iteration 3
link: https://www.cmjornal.pt/boa-vida/detalhe/capital-do-natal-chega-a-oeiras
iteration 4
link: https://www.cmjornal.pt/boa-vida/detalhe/capital-do-natal-chega-a-oeiras?ref=HP_Africa_BlocoBoaVida&act=0&est=Aberto
iteration 5
link: https://www.cmjornal.pt/boa-vida/detalhe/capital-do-natal-chega-a-oeiras?ref=HP_Africa_BlocoBoaVida&act=0&est=Aberto
iteration 6
link: https://www.cmjornal.pt/boa-vida/detalhe/capital-do-natal-chega-a-oeiras?ref=HP_BlocoBoaVida&act=0&est=Aberto
iteration 7
link: https://www.cmjornal.pt/boa-vida/detalhe/capital-do-natal-chega-a-oeiras?ref=HP_BlocoBoaVida&act=0&est=Aberto
iteration 8
link: https://www.cmjornal.pt/boa-vida/detalhe/capital-do-natal-chega-a-oeiras?ref=HP_Africa_BlocoBoaVida


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


iteration 868
link: https://www.cmjornal.pt/mundo/africa/detalhe/marcelo-rebelo-de-sousa-chega-hoje-a-abidjan-para-visita-de-estado


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


iteration 869
link: https://www.cmjornal.pt/mundo/africa/detalhe/marcelo-rebelo-de-sousa-chega-hoje-a-abidjan-para-visita-de-estado?ref=HP_Africa_OutrasNoticias1&act=0&est=Aberto
iteration 870
link: https://www.cmjornal.pt/mundo/africa/detalhe/marcelo-rebelo-de-sousa-chega-hoje-a-abidjan-para-visita-de-estado?ref=HP_Africa_OutrasNoticias1


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


iteration 871
link: https://www.cmjornal.pt/mundo/amp/acao-de-rua-contra-violencia-sobre-mulheres-chega-em-portugues-a-paris
iteration 872
link: https://www.cmjornal.pt/mundo/amp/ajuda-humanitaria-chega-no-dia-23-a-venezuela
iteration 873
link: https://www.cmjornal.pt/mundo/amp/ajuda-humanitaria-chega-no-dia-23-a-venezuela
iteration 874
link: https://www.cmjornal.pt/mundo/amp/boeing-chega-a-acordo-e-paga-cerca-de-um-milhao-de-euros-a-familias-das-vitimas-da-queda-de-aviao-da-lion-air
iteration 875
link: https://www.cmjornal.pt/mundo/amp/boeing-chega-a-acordo-e-paga-cerca-de-um-milhao-de-euros-a-familias-das-vitimas-da-queda-de-aviao-da-lion-air
iteration 876
link: https://www.cmjornal.pt/mundo/amp/bolsonaro-chega-irritado-ao-japao-para-reuniao-do-g20-e-responde-mal-a-merkel
iteration 877
link: https://www.cmjornal.pt/mundo/amp/bolsonaro-chega-irritado-ao-japao-para-reuniao-do-g20-e-responde-mal-a-merkel
iteration 878
link: https://www.cmjornal.pt/mundo/amp/carreira-de-britney-spears-ch

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


iteration 2140
link: https://www.cmjornal.pt/sociedade/amp/movimento-giving-tuesday-chega-hoje-a-portugal
iteration 2141
link: https://www.cmjornal.pt/sociedade/amp/tap-chega-a-17-cidades-africanas-com-voo-lisboa-conacri-a-partir-de-3-de-julho
iteration 2142
link: https://www.cmjornal.pt/sociedade/amp/tap-chega-a-17-cidades-africanas-com-voo-lisboa-conacri-a-partir-de-3-de-julho
iteration 2143
link: https://www.cmjornal.pt/sociedade/amp/temperatura-vai-subir-maxima-chega-aos-26-graus-durante-o-fim-de-semana
iteration 2144
link: https://www.cmjornal.pt/sociedade/amp/temperatura-vai-subir-maxima-chega-aos-26-graus-durante-o-fim-de-semana
iteration 2145
link: https://www.cmjornal.pt/sociedade/amp/temperatura-vai-subir-maxima-chega-aos-26-graus-durante-o-fim-de-semana
iteration 2146
link: https://www.cmjornal.pt/sociedade/amp/tempo--massa-de-ar-frio-chega-a-portugal-temperaturas-descem-ate-10-graus-ja-amanha
iteration 2147
link: https://www.cmjornal.pt/sociedade/amp/tempo--massa-de-ar-frio

In [ ]:
8519